In [1]:
import easyocr

# Create an EasyOCR Reader instance
reader = easyocr.Reader(['en'])  # Specify the language(s)

# Perform OCR
def perform_ocr_easyocr(image_path):
    try:
        result = reader.readtext(image_path)
        extracted_text = " ".join([text[1] for text in result])  # Extract recognized text
        return extracted_text
    except Exception as e:
        print(f"Error during OCR: {e}")
        return None

# Example usage
image_path = "resources/test.png"  # Path to your image

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
text = perform_ocr_easyocr(image_path)
print("Extracted Text:", text)

Extracted Text: Q01. (25 Points) Locate the relative extrema for this function y =f(x) = 12 Answer: X-x?


In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader_pdf = PyPDFLoader('resources/mother-doc.pdf')
rag_data = loader_pdf.load()

In [3]:
rag_data

[Document(metadata={'source': 'resources/mother-doc.pdf', 'page': 0}, page_content='The Project Gutenberg EBook of Calculus Made Easy, by Silvanus Thompson\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\nTitle: Calculus Made Easy\nBeing a very-simplest introduction to those beautiful\nmethods which are generally called by the terrifying names\nof the Differentia\nAuthor: Silvanus Thompson\nRelease Date: October 9, 2012 [eBook #33283]\nMost recently updated: November 18, 2021\nLanguage: English\nCharacter set encoding: UTF-8\n*** START OF THIS PROJECT GUTENBERG EBOOK CALCULUS MADE EASY ***')

In [4]:
%pip install PyMuPDF

   ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
   ----------- ---------------------------- 4.7/16.3 MB 28.6 MB/s eta 0:00:01
   ---------------------- ----------------- 9.2/16.3 MB 23.8 MB/s eta 0:00:01
   -------------------------------- ------- 13.1/16.3 MB 22.2 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.3 MB 21.8 MB/s eta 0:00:01
   ---------------------------------------- 16.3/16.3 MB 20.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.
    """
    doc = fitz.open(pdf_path)
    extracted_text = ""
    for page_num in range(len(doc)):
        page = doc[page_num]
        extracted_text += page.get_text()
    return extracted_text

# Extract text from "Calculus Made Easy"
pdf_text = extract_text_from_pdf("resources/mother-doc.pdf")

In [3]:
pdf_text

'CHAPTER I.\nTO DELIVER YOU FROM THE PRELIMINARY\nTERRORS.\nThe preliminary terror, which chokes off most fifth-form boys from\neven attempting to learn how to calculate, can be abolished once for\nall by simply stating what is the meaning—in common-sense terms—of\nthe two principal symbols that are used in calculating.\nThese dreadful symbols are:\n(1) d which merely means “a little bit of.”\nThus dx means a little bit of x; or du means a little bit of u. Or-\ndinary mathematicians think it more polite to say “an element of,”\ninstead of “a little bit of.” Just as you please. But you will find that\nthese little bits (or elements) may be considered to be indefinitely small.\n(2)\nZ\nwhich is merely a long S, and may be called (if you like) “the\nsum of.”\nThus\nZ\ndx means the sum of all the little bits of x; or\nZ\ndt means\nthe sum of all the little bits of t. Ordinary mathematicians call this\nsymbol “the integral of.” Now any fool can see that if x is considered\nas made up of a l

In [4]:
cleaned_text = pdf_text.replace('\n', ' ').strip()


In [5]:
cleaned_text

'CHAPTER I. TO DELIVER YOU FROM THE PRELIMINARY TERRORS. The preliminary terror, which chokes off most fifth-form boys from even attempting to learn how to calculate, can be abolished once for all by simply stating what is the meaning—in common-sense terms—of the two principal symbols that are used in calculating. These dreadful symbols are: (1) d which merely means “a little bit of.” Thus dx means a little bit of x; or du means a little bit of u. Or- dinary mathematicians think it more polite to say “an element of,” instead of “a little bit of.” Just as you please. But you will find that these little bits (or elements) may be considered to be indefinitely small. (2) Z which is merely a long S, and may be called (if you like) “the sum of.” Thus Z dx means the sum of all the little bits of x; or Z dt means the sum of all the little bits of t. Ordinary mathematicians call this symbol “the integral of.” Now any fool can see that if x is considered as made up of a lot of little bits, each 

In [29]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


 #### cleaned text will be used for fine tuning using LORA + adaptive layers approach (part of training_pipeline)

In [ ]:
# tokenizing the text

In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
inputs = tokenizer(cleaned_text, return_tensors='pt', truncation=True, max_length=1024)

In [7]:
print(inputs['input_ids'][:5])

tensor([[41481,   314,    13,  ...,   583,  1247,    13]])


In [35]:
%pip install --upgrade peft


Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from peft import LoRAConfig, get_peft_model
import torch

# Load the GPT2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Freeze the base GPT model (no parameters will be updated)
for param in model.parameters():
    param.requires_grad = False

# Define LoRA configuration for low-rank adaptation
lora_config = LoRAConfig(
    r=8,  # Rank of the low-rank matrices (adjust for performance)
    alpha=16,  # Scaling factor for the low-rank matrices
    dropout=0.1,  # Dropout rate
    target_modules=["attn.c_attn", "mlp.c_fc"]  # Target transformer layers for LoRA
)

ImportError: cannot import name 'LoRAConfig' from 'peft' (f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\peft\__init__.py)